# Projet -  Assistant d'Analyse de Documents

## **1. Objectifs**

- **Extraire du texte** à partir de divers formats de documents (ex. : images, PDF, documents numérisés) en utilisant des outils OCR et de parsing.
- **Traiter et structurer le texte extrait** pour en faire une base de connaissances interrogeable et consultable.
- **Développer un assistant IA** capable de :
  - Répondre aux questions des utilisateurs sur la base du contenu extrait.
  - Résumer des documents volumineux.
- **Fournir une interface conviviale** pour le téléversement de documents et l’interaction avec l’assistant (optionnel).


## **2. Etapes de mise en œuvre**

### **2.1. Extraction de texte**

In [1]:
!pip install pdfplumber
!pip install pytesseract
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 23.1 MB/s eta 0:00:00


Importation de biblithèque

In [2]:
import pdfplumber
import pytesseract
from PIL import Image
import re
from transformers import pipeline

In [3]:
!pip install pillow

Si nous voulons extraire le français de la photo, nous utiliserons cette bibliothèque(tesseract-ocr-fra)

on peut utiliser pytesseract.image_to_string(Image.open(image_path), lang="fra")

In [4]:
!sudo apt install tesseract-ocr libtesseract-dev
!sudo apt-get install tesseract-ocr-fra

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libarchive-dev libleptonica-dev tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  libarchive-dev libleptonica-dev libtesseract-dev tesseract-ocr
  tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 6 newly installed, 0 to remove and 29 not upgraded.
Need to get 8,560 kB of archives.
After this operation, 31.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libarchive-dev amd64 3.6.0-1ubuntu1.3 [581 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libleptonica-dev amd64 1.82.0-3build1 [1,562 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libtesseract-dev amd64 4.1.1-2.1build1 [1,600 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:5 http://arc

Info: https://pypi.org/project/pdfplumber/

### Fonction pour extraire le texte d'un PDF

In [5]:
def extract_text_from_pdf(pdf_path):
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text() + "\n"
    return text

Info: https://pypi.org/project/pytesseract/

### Fonction pour extraire le texte d'une image (OCR)

In [6]:
def extract_text_from_image(image_path):
    return pytesseract.image_to_string(Image.open(image_path))

### Test: Extraire du texte d'une image et d'un fichier PDF

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
!pwd

/content


In [9]:
!ls

drive  sample_data


In [10]:
pdf_path = './drive/MyDrive/Projet_AI/ressources/Generative_AI-5.pdf'
pdf_text = extract_text_from_pdf(pdf_path)
print(pdf_text)

Introduction a` l’IA G´en´erative
Projet 2 : Assistant d’Analyse de Documents
2024 - 2025
Issam Falih
Le projet “Assistant d’Analyse de Documents” vise `a d´evelopper un
syst`emed’IAcapabled’extraireetdetraiterdutexte`apartirdediversformats
de documents, notamment des images, des fichiers PDF et des fichiers texte
structur´es. Le texte extrait sera utilis´e pour construire un assistant interactif
capable de r´esumer des documents, r´epondre aux questions des util-
isateurs et mettre en ´evidence les informations cl´es. Ce projet int`egre
des techniques de Reconnaissance Optique de Caract`eres (OCR), de
Traitement du Langage Naturel (NLP) et d’IA G´en´erative.
1 Objectifs
• Extraire du texte `a partir de divers formats de documents (ex. : images,
PDF, documents num´eris´es) en utilisant des outils OCR et de parsing.
• Traiter et structurer le texte extrait pour en faire une base de connais-
sances interrogeable et consultable.
• D´evelopper un assistant IA capable de :
– R´epondre aux q

In [11]:
image_path = './drive/MyDrive/Projet_AI/ressources/Restaurant-slogans.jpg'
image_text = extract_text_from_image(image_path)
print(image_text)

 

ad

.

OBESE

 

+Stogars

BUSINESS SLOGANS IDEAS

What's your idea?- Coca Cola

You're in good hands.- Allstate Insurance Company
The best a man can get.- Gillette

pushing boundaries.- Nike

“The pause that refreshes"- Coca Cola

“Good to the last drop"- Maxwell House Coffee

“The Breakfast of Champions"- Wheaties

“A diamond is forever"- De Beers

“Think outside the bun"- Taco Bell

19."Finger-lickin' good"- KFC

 

 



### **2.2. Prétraitement du texte**

### Nettoyer et prétraiter le texte extrait (ex. : suppression du bruit, tokenisation)

L'analyse OCR ou PDF peut introduire des espaces supplémentaires, des sauts de ligne et des caractères spéciaux. Ces bruits de données affecteront les tâches NLP ultérieures (telles que la synthèse et la réponse aux questions).
Utilisez re.sub() pour remplacer rapidement les caractères indésirables.

In [12]:
!python -m spacy download fr_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 73.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


**Informations sur les demandes de renseignements**

1. Préserver les lettres majuscules et minuscules, les lettres latines accentuées courantes, les chiffres, la ponctuation de base, les apostrophes, les espaces blancs en annulant le jeu de caractères.【   texte = re.sub(r"[^a-zA-ZÀ-ÿ0-9.,;:!?’'\s]", "", texte)】Y compris ce symbole **'** « par exemple dans l'introduction »

2. Le modèle fr_core_news_sm, entraîné ici pour le français, traite correctement les participes spécifiques au français

In [13]:
import re
import nltk
import spacy

nltk.download('punkt')
nlp = spacy.load('fr_core_news_sm')

def nettoyer_texte(texte):
    # Supprimer les caractères spéciaux et espaces inutiles
    texte = re.sub(r"[^a-zA-ZÀ-ÿ0-9.,;:!-?’'\s]", "", texte)
    texte = re.sub(r'\s+', ' ', texte).strip()
    # Tokenisation avec spaCy
    tokens = [token.text for token in nlp(texte)]
    return " ".join(tokens)

pdf_text_cleaned = nettoyer_texte(pdf_text)
print(pdf_text_cleaned)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Introduction a l’ IA Generative Projet 2 : Assistant d’ Analyse de Documents 2024 - 2025 Issam Falih Le projet Assistant d’ Analyse de Documents vise a developper un systemed’ IAcapabled’ extraireetdetraiterdutexteapartirdediversformats de documents , notamment des images , des fichiers PDF et des fichiers texte structures . Le texte extrait sera utilise pour construire un assistant interactif capable de resumer des documents , repondre aux questions des util- isateurs et mettre en evidence les informations cles . Ce projet integre des techniques de Reconnaissance Optique de Caracteres ( OCR ) , de Traitement du Langage Naturel ( NLP ) et d’ IA Generative . 1 Objectifs Extraire du texte a partir de divers formats de documents ( ex . : images , PDF , documents numerises ) en utilisant des outils OCR et de parsing . Traiter et structurer le texte extrait pour en faire une base de connais- sances interrogeable et consultable . Developper un assistant IA capable de : Repondre aux questions

### Segmenter le texte en composants logiques tels que titres, paragraphes et tableaux.

tableaux：j'utilise le méthode générative, parce que il y a pas de tabeaux dans le fichier pdf

In [14]:
def detect_titles_paragraphs_tables_v2(text):
    segments = {"Titres": [], "Paragraphes": [], "Tableaux": []}
    current_section = "Paragraphes"

    lines = text.split(". ")  # Les points d'arrêt sont placés en premier pour éviter les sauts de ligne.

    for line in lines:
        line = line.strip()
        if not line:
            continue
        if re.match(r"^\d+(\.\d+)?\s*[A-ZÀ-Ú].+", line):
            segments["Titres"].append(line)
            current_section = "Titres"
        # 2. Formulaire de détection (basé sur des espaces multiples)
        elif re.search(r"\s{4,}", line):  #4 et plus d'espaces, éventuellement des tableaux
            segments["Tableaux"].append(line)
            current_section = "Tableaux"
        else:
            segments["Paragraphes"].append(line)
            current_section = "Paragraphes"

    return segments

segments = detect_titles_paragraphs_tables_v2(pdf_text_cleaned)
segments


{'Titres': ['1 Objectifs Extraire du texte a partir de divers formats de documents ( ex',
  '2 Etapes de mise en uvre 2.1 Extraction de texte Extraire du texte depuis des fichiers PDF en utilisant des bibliotheques comme PyPDF2 ou pdfplumber',
  '1 Gerer les cas complexes , comme les textes manuscrits ou les donnees tabulaires , grace a des techniques avancees d’ OCR',
  '2.2 Pretraitement du texte Nettoyer et pretraiter le texte extrait ( ex',
  '2.3 Fonctionnalites de l’ assistant Fournir des reponses precises aux questions des utilisateurs',
  '2.4 Interface utilisateur DevelopperuneinterfacewebenutilisantdesframeworkscommeStream- lit',
  '3 Extensions optionnelles Support multilingue : permettrel’ extractiondetexteetlareponseaux questions dans plusieurs langues'],
 'Paragraphes': ['Introduction a l’ IA Generative Projet 2 : Assistant d’ Analyse de Documents 2024 - 2025 Issam Falih Le projet Assistant d’ Analyse de Documents vise a developper un systemed’ IAcapabled’ extraireetdetra

### Générer les embeddings à partir du texte nettoyé
Ensuite, on peux utiliser Sentence-Transformers pour générer des embeddings à partir du texte nettoyé.

In [15]:
from sentence_transformers import SentenceTransformer
import numpy as np

model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Générer l'intégration et créer l'index FAISS
def create_embeddings_and_index(segments):
    # Créer des liens pour chaque paragraphe (y compris les titres, les paragraphes et les tableaux)
    embeddings = []
    for section, lines in segments.items():
        for line in lines:
            embedding = model.encode(line)
            embeddings.append(embedding)

    embeddings = np.array(embeddings).astype('float32')

    # Création de l'index FAISS
    index = faiss.IndexFlatL2(embeddings.shape[1])
    index.add(embeddings)

    return index, embeddings

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.51k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

**Recherche avec FAISS**

In [16]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 34.8 MB/s eta 0:00:00


In [17]:
import faiss
def search_faiss(query, index, top_k=5):
    # Convertir le texte de la requête en un texte intégré
    query_embedding = model.encode([query]).astype('float32')

    # Recherche des articles k les plus similaires
    distances, indices = index.search(query_embedding, top_k)
    return distances, indices

# Création d'emboîtements et d'index
index, embeddings = create_embeddings_and_index(segments)

**Test**

In [18]:
query = "Système d'IA"
distances, indices = search_faiss(query, index)

# Afficher les résultats de la recherche
print("Top 5 résultats pour la requête '{}':".format(query))
for i in range(len(indices[0])):
    print(f"{i+1}. {segments['Titres'][indices[0][i]] if indices[0][i] < len(segments['Titres']) else ''} {segments['Paragraphes'][indices[0][i] - len(segments['Titres'])] if indices[0][i] >= len(segments['Titres']) else ''} - Distance: {distances[0][i]}")


Top 5 résultats pour la requête 'Système d'IA':
1.  Introduction a l’ IA Generative Projet 2 : Assistant d’ Analyse de Documents 2024 - 2025 Issam Falih Le projet Assistant d’ Analyse de Documents vise a developper un systemed’ IAcapabled’ extraireetdetraiterdutexteapartirdediversformats de documents , notamment des images , des fichiers PDF et des fichiers texte structures - Distance: 44.258426666259766
2.  Mecanismedefeedback : implementerunsystemederetourutilisateur pour affiner les reponses de l’ assistant au fil du temps - Distance: 54.66343307495117
3. 2.3 Fonctionnalites de l’ assistant Fournir des reponses precises aux questions des utilisateurs  - Distance: 55.36997985839844
4.  Traiter et structurer le texte extrait pour en faire une base de connais- sances interrogeable et consultable - Distance: 56.07222366333008
5. 1 Gerer les cas complexes , comme les textes manuscrits ou les donnees tabulaires , grace a des techniques avancees d’ OCR  - Distance: 57.190513610839844


### 2.3 Fonctionnalités de l’assistant

In [ ]:
# import os
# from dotenv import load_dotenv

# load_dotenv()
# openai_key = os.getenv("OPENAI_API_KEY")

In [ ]:
# import openai

# openai.api_key = openai_key
# def reponse_question(question, document):
#     prompt = f"Répondez à la question suivante en fonction du contenu du document ci-dessous :\n\nDocument : {document}\n\nQuestion : {question}\nRéponse :"

#     # Utiliser le modèle GPT-4 ou GPT-3.5 selon ta préférence
#     response = openai.ChatCompletion.create(
#         model="gpt-4",  # Ou tu peux utiliser "gpt-3.5-turbo"
#         messages=[
#             {"role": "system", "content": "Vous êtes un assistant utile."},
#             {"role": "user", "content": prompt}
#         ]
#     )

#     return response['choices'][0]['message']['content']

# # Exemple d'utilisation

# question = "Que fait l'IA générative ?"

# reponse = reponse_question(question, pdf_text_cleaned)
# print(reponse)


Cela pourrait être fait ici en utilisant la clé openai utilisée dans le cours tp, mais nous pouvons baser la fonctionnalité Q&A sur l'intégration de texte et l'index FAISS créés précédemment.

In [20]:
def answer_question(question, index, segments, model, top_k=3):
    distances, indices = search_faiss(question, index, top_k)

    context = ""
    for i in range(top_k):
        if indices[0][i] < len(segments['Titres']):
            context += segments['Titres'][indices[0][i]] + " "
        else:
            context += segments['Paragraphes'][indices[0][i] - len(segments['Titres'])] + " "

    qa_pipeline = pipeline("question-answering", model="illuin/camembert-base-fquad")
    answer = qa_pipeline(question=question, context=context)

    return answer['answer']

Utiliser le modèle de résumé de Hugging Face :

In [21]:
def generate_summary(text, max_length=150):
    summarizer = pipeline("summarization", model="plguillou/t5-base-fr-sum-cnndm")
    summary = summarizer(text, max_length=max_length, min_length=30, do_sample=False)
    return summary[0]['summary_text']

Nous pouvons combiner la reconnaissance d'entités nommées (NER) et l'extraction de mots-clés.

In [22]:
def extract_key_information(text):
    nlp = spacy.load("fr_core_news_sm")
    doc = nlp(text)

    entities = {}
    for ent in doc.ents:
        if ent.label_ not in entities:
            entities[ent.label_] = []
        entities[ent.label_].append(ent.text)

    # Extraction de mots-clés
    keywords = []
    for token in doc:
        if token.pos_ in ["NOUN", "PROPN"] and not token.is_stop:
            keywords.append(token.text)

    return {
        "entities": entities,
        "keywords": list(set(keywords))[:10]
    }

**Test**

In [23]:
# Tester la fonction Q&A
question = "Quel est l'objectif principal de ce projet?"
answer = answer_question(question, index, segments, model)
print(f"Question: {question}")
print(f"Réponse: {answer}\n")

# Fonction de résumé des tests
summary = generate_summary(pdf_text_cleaned)
print("Résumé du document:")
print(summary + "\n")

# Test d'extraction d'informations clés
key_info = extract_key_information(pdf_text_cleaned[:500])
print("Informations clés extraites:")
print(key_info)

config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/443M [00:00<?, ?B/s]

Some weights of the model checkpoint at illuin/camembert-base-fquad were not used when initializing CamembertForQuestionAnswering: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing CamembertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/811k [00:00<?, ?B/s]

...

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/31.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Device set to use cpu
Token indices sequence length is longer than the specified maximum sequence length for this model (815 > 512). Running this sequence through the model will result in indexing errors


Résumé du document:
Le projet IA Generative vise à intégrer des techniques de Reconnaissance Optique de Caracteres ( OCR ) et de Traitement du Langage Naturel ( NLP ). Le texte extrait sera utilise pour construire un assistant interactif capable de resumer des documents en utilisant des outils comme PyPDF2 ou pdfplumber. Fonctionnalités de l'assistant Fournir des reponses précises aux questions des utilisateurs.

Informations clés extraites:
{'entities': {'LOC': ['Introduction'], 'ORG': ['IA Generative Projet 2'], 'MISC': ['Analyse de Documents 2024', 'Issam Falih', 'Analyse de Documents', 'IAcapabled', 'PDF']}, 'keywords': ['IAcapabled', 'images', 'questions', 'Generative', 'Issam', 'systemed', 'projet', 'vise', 'fichiers', 'IA']}
